In [30]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [238]:
with open('./res/lab1.json') as lab1_file:
    data = json.load(lab1_file)
chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']
chain

[['I', 0, 1],
 ['W', 12, 1],
 ['I', 1, 1],
 ['O', 6, 1],
 ['I', 2, 1],
 ['R', 2, 1],
 ['W', 5, 2],
 ['R', 0, 2],
 ['I', 3, 2],
 ['R', 2, 2],
 ['I', 4, 3],
 ['W', 12, 3],
 ['I', 5, 3],
 ['O', 8, 3],
 ['I', 6, 3],
 ['O', 7, 3],
 ['I', 7, 3],
 ['R', 2, 3],
 ['I', 8, 4],
 ['W', 13, 4]]

In [239]:
def get_priority(operation):
    priorities = [
        ['(', '[', 'АЭМ', 'Ф', 'IF'],
        [')', ']', ',', ';', 'THEN', 'ELSE'],
        ['=', 'goto'],
        ['||'],
        ['&&'],
        ['!'],
        ['<', '>', '<=', '>=', '==', '!='],
        ['+', '-'],
        ['*', '/'],
        ['^'],
        [':']
    ]
    i = 0
    while operation not in priorities[i]:
        i+=1
        if i > len(priorities) - 1:
            return False
    return i

In [240]:
def get_token_name(token):
    global service_words, operations, separators, constants, identifiers
    codes = ['W', 'O', 'R', 'C', 'I']
    tables = [service_words, operations, separators, constants, identifiers]
    if token[0] in ['C', 'I']:
        return tables[codes.index(token[0])][token[1]]['value']
    return tables[codes.index(token[0])][token[1]]

In [241]:
def get_token_type(token):
    global service_words, operations, separators, constants, identifiers
    token_name = get_token_name(token)
    if token_name in service_words:
        return 'service_word'
    if token_name in operations:
        return 'operation'
    if token_name in separators:
        return 'separator'
    if token[0] == 'C':
        return constants[token[1]]['type']
    if token[0] == 'I':
        return identifiers[token[1]]['type']
    return False

In [247]:
stack = []
result = []
AEM_counter = None
F_counter = None

identifier_context = False

for token in chain:
    token_name = get_token_name(token)
    token_type = get_token_type(token)
    previous_stack = list(stack)
#     print(token, token_name, token_type)
    
    if token_type != 'separator' or token_name in ['(', ')', '[', ']', ',', ':', ';']:
        if token_type == 'identifier':
            identifier_context = True
            
#         ";" handling
        if token_name == ';':
            while stack:
                result.append(stack.pop())
            

#         Ordinary expression handling
        if token_type in ['identifier', 'integer', 'string', 'float']:
            result.append(token_name)
            
        if token_type in ['operation'] or token_name in ['=', 'goto', ':']:
            if not stack:
                stack.append(token_name)
            else:        
                while stack and get_priority(stack[-1]) > get_priority(token_name):
                    result.append(stack.pop())
                else:
                    stack.append(token_name)
                    
#         Brackets handling
        if token_name == '(' and not identifier_context:
            stack.append(token_name)
        if token_name == ')' and not F_counter:
            while stack[-1] != '(':
                print(stack)
                result.append(stack.pop())
            stack.pop()
            
#         Arrays handling
        if token_name == '[':
            stack.append('АЭМ')
            AEM_counter = 2
            
        if token_name == ',' and AEM_counter:
            while stack[-1] != 'АЭМ':
                result.append(stack.pop())
            AEM_counter += 1
            
        if token_name == ']':
            while stack[-1] != 'АЭМ':
                result.append(stack.pop())
            result.append(str(AEM_counter) + stack.pop())
            AEM_counter = None
            
#         Functions handling
        if token_name == '(' and identifier_context:
            stack.append('Ф')
            F_counter = 1
            
        if token_name == ',' and F_counter:
            while stack[-1] != 'Ф':
                result.append(stack.pop())
            F_counter += 1
        
        if token_name == ')' and F_counter:
            F_counter += 1
            while stack[-1] != 'Ф':
                result.append(stack.pop())
            result.append(str(F_counter) + stack.pop())
            F_counter = None

        print(str(token_name), Fore.WHITE + str(token_type) + Style.RESET_ALL)
        print('Stack:', previous_stack, '->', stack)
        print('Result:', result)
        print()
        
        if token_type != 'identifier':
            identifier_context = False
            
    
previous_stack = list(stack)
while stack:
    result.append(stack.pop())

print(Fore.RED + 'END' + Style.RESET_ALL)
print('Stack:', previous_stack, '->', stack)
print('Result:', result, Fore.BLUE)

for i in result:
    print(i, end='')

r1 identifier
Stack: [] -> []
Result: ['r1']

= service_word
Stack: [] -> ['=']
Result: ['r1']

a identifier
Stack: ['='] -> ['=']
Result: ['r1', 'a']

+ operation
Stack: ['='] -> ['=', '+']
Result: ['r1', 'a']

b identifier
Stack: ['=', '+'] -> ['=', '+']
Result: ['r1', 'a', 'b']

; separator
Stack: ['=', '+'] -> []
Result: ['r1', 'a', 'b', '+', '=']

goto service_word
Stack: [] -> ['goto']
Result: ['r1', 'a', 'b', '+', '=']

m1 identifier
Stack: ['goto'] -> ['goto']
Result: ['r1', 'a', 'b', '+', '=', 'm1']

; separator
Stack: ['goto'] -> []
Result: ['r1', 'a', 'b', '+', '=', 'm1', 'goto']

r2 identifier
Stack: [] -> []
Result: ['r1', 'a', 'b', '+', '=', 'm1', 'goto', 'r2']

= service_word
Stack: [] -> ['=']
Result: ['r1', 'a', 'b', '+', '=', 'm1', 'goto', 'r2']

c identifier
Stack: ['='] -> ['=']
Result: ['r1', 'a', 'b', '+', '=', 'm1', 'goto', 'r2', 'c']

* operation
Stack: ['='] -> ['=', '*']
Result: ['r1', 'a', 'b', '+', '=', 'm1', 'goto', 'r2', 'c']

d identifier
Stack: ['=', '*'